O Princípio do Aberto/Fechado (OCP) é o segundo princípio do SOLID e afirma que entidades de software (classes, módulos, funções, etc.) devem estar abertas para extensão, mas fechadas para modificação. Isso significa que você deve ser capaz de adicionar novas funcionalidades sem alterar o código existente. https://programadriano.medium.com/princ%C3%ADpios-do-solid-com-typescript-2c6a4911242d


In [1]:
abstract class Investimento {
    constructor(public nome: string, public valorInvestido: number) {}

    abstract calcularRetornoAnual(): number;

    descrever(): string {
        return `Investimento: ${this.nome}, Valor Investido: R$ ${this.valorInvestido}`;
    }
}

class Acao extends Investimento {
    constructor(
        nome: string,
        valorInvestido: number,
        public quantidade: number,
        public precoPorAcao: number,
        public dividendoAnual: number
    ) {
        super(nome, valorInvestido);
    }

    calcularRetornoAnual(): number {
        return this.quantidade * this.dividendoAnual;
    }

    descrever(): string {
        return `${super.descrever()}, Quantidade: ${this.quantidade}, Preço por Ação: R$ ${this.precoPorAcao}, Dividendo Anual: R$ ${this.dividendoAnual}`;
    }
}

class Titulo extends Investimento {
    constructor(
        nome: string,
        valorInvestido: number,
        public taxaDeJurosAnual: number
    ) {
        super(nome, valorInvestido);
    }

    calcularRetornoAnual(): number {
        return this.valorInvestido * (this.taxaDeJurosAnual / 100);
    }

    descrever(): string {
        return `${super.descrever()}, Taxa de Juros Anual: ${this.taxaDeJurosAnual}%`;
    }
}

class Imovel extends Investimento {
    constructor(
        nome: string,
        valorInvestido: number,
        public aluguelMensal: number
    ) {
        super(nome, valorInvestido);
    }

    calcularRetornoAnual(): number {
        return this.aluguelMensal * 12;
    }

    descrever(): string {
        return `${super.descrever()}, Aluguel Mensal: R$ ${this.aluguelMensal}`;
    }
}


InvestimentoService

In [2]:

class InvestimentoService {
    
    calcularRetornoAnual(investimento: Investimento): number {
        if (investimento instanceof Acao) {
            return investimento.quantidade * investimento.dividendoAnual;
        } else if (investimento instanceof Titulo) {
            return investimento.valorInvestido * (investimento.taxaDeJurosAnual / 100);
        } else if (investimento instanceof Imovel) {
            return investimento.aluguelMensal * 12;
        }
        throw new Error("Tipo de investimento desconhecido");
    }

    descrever(investimento: Investimento): string {
        if (investimento instanceof Acao) {
            return `Ação: ${investimento.nome}, Quantidade: ${investimento.quantidade}, Preço por Ação: R$ ${investimento.precoPorAcao}, Dividendo Anual: R$ ${investimento.dividendoAnual}`;
        } else if (investimento instanceof Titulo) {
            return `Título: ${investimento.nome}, Valor Investido: R$ ${investimento.valorInvestido}, Taxa de Juros Anual: ${investimento.taxaDeJurosAnual}%`;
        } else if (investimento instanceof Imovel) {
            return `Imóvel: ${investimento.nome}, Valor Investido: R$ ${investimento.valorInvestido}, Aluguel Mensal: R$ ${investimento.aluguelMensal}`;
        }
        throw new Error("Tipo de investimento desconhecido");
    }
}

In [3]:
const acao = new Acao("Empresa X", 5000, 100, 50, 5);
const titulo = new Titulo("Título Y", 10000, 7);
const imovel = new Imovel("Apartamento Z", 300000, 1500);

const investimentoService = new InvestimentoService();
console.log(investimentoService.descrever(acao));
console.log(`Retorno anual: R$ ${investimentoService.calcularRetornoAnual(acao)}`);

Retorno anual: R$ 500


In [4]:
console.log(investimentoService.descrever(titulo));
console.log(`Retorno anual: R$ ${investimentoService.calcularRetornoAnual(titulo)}`);

Retorno anual: R$ 700.0000000000001


In [5]:
console.log(investimentoService.descrever(imovel));
console.log(`Retorno anual: R$ ${investimentoService.calcularRetornoAnual(imovel)}`);

Retorno anual: R$ 18000


Refatorando

In [6]:
interface IInvestimentoService<T extends Investimento> {
    calcularRetornoAnual(investimento: T): number;
    descrever(investimento: T): string;
}

In [7]:

class AcaoService implements IInvestimentoService<Acao> {
    calcularRetornoAnual(acao: Acao): number {
        return acao.calcularRetornoAnual();
    }

    descrever(acao: Acao): string {
        return acao.descrever();
    }
}

class TituloService implements IInvestimentoService<Titulo> {
    calcularRetornoAnual(titulo: Titulo): number {
        return titulo.calcularRetornoAnual();
    }

    descrever(titulo: Titulo): string {
        return titulo.descrever();
    }
}

class ImovelService implements IInvestimentoService<Imovel> {
    calcularRetornoAnual(imovel: Imovel): number {
        return imovel.calcularRetornoAnual();
    }

    descrever(imovel: Imovel): string {
        return imovel.descrever();
    }
}

Registrando [DI]

In [8]:
class Container {
    private services = new Map<string, any>();

    register<T>(name: string, implementation: new (...args: any[]) => T): void {
        this.services.set(name, implementation);
    }

    resolve<T>(name: string): T {
        const implementation = this.services.get(name);
        if (!implementation) {
            throw new Error(`Service not found: ${name}`);
        }
        return new implementation();
    }
}

const container = new Container();
container.register<IInvestimentoService<Acao>>("AcaoService", AcaoService);
container.register<IInvestimentoService<Titulo>>("TituloService", TituloService);
container.register<IInvestimentoService<Imovel>>("ImovelService", ImovelService);

Implementação

In [9]:
const acao = new Acao("Empresa X", 5000, 100, 50, 5);
const titulo = new Titulo("Título Y", 10000, 7);
const imovel = new Imovel("Apartamento Z", 300000, 1500);

const acaoService = container.resolve<IInvestimentoService<Acao>>("AcaoService");
const tituloService = container.resolve<IInvestimentoService<Titulo>>("TituloService");
const imovelService = container.resolve<IInvestimentoService<Imovel>>("ImovelService");

In [10]:
console.log(acaoService.descrever(acao));
console.log(`Retorno anual: R$ ${acaoService.calcularRetornoAnual(acao)}`);

Retorno anual: R$ 500


In [11]:
console.log(tituloService.descrever(titulo));
console.log(`Retorno anual: R$ ${tituloService.calcularRetornoAnual(titulo)}`);

Retorno anual: R$ 700.0000000000001


In [12]:
console.log(imovelService.descrever(imovel));
console.log(`Retorno anual: R$ ${imovelService.calcularRetornoAnual(imovel)}`);

Retorno anual: R$ 18000
